In addition to the test pattern functionality that the experiment in this branch explores, this notebook explores another potential novel usage of the model parameters: adjusting the CRC survival rate for different demographic groups.

Some of the scenarios we want to simulate in the future involve differing CRC burdens among demographic groups. One of those differences is that black populations have lower CRC survival rates than white populations. We want to be able to account for such differences in simulations, but the model currently uses a single set of parameters controlling CRC survival. 

```
"mean_duration_clin1_dead": 135,
"mean_duration_clin2_dead": 51,
"mean_duration_clin3_dead": 19,
"mean_duration_clin4_dead": 2.7,
```
When an agent is diagnosed with clinical CRC, the time to their CRC death event is drawn from an exponential distribution. These parameters, along with the cancer's stage at diagnosis, control the mean of those distributions.

The purpose of this notebook is to determine whether we can alter those parameters to attain given 5-year CRC survival rates by stage without recalibration. When time to death has an exponential distribution, there is a mathematical relationship between the survival rate and the mean of the distribution:
```
μ = -n / ln(P(t > n))
```
Where `μ` is the mean of the exponential distribution, `n` is the time at which survival is assessed (e.g., 5 for a 5-year survival rate), and `P(t > n)` is the survival rate. 

We want to see if this relationship actually holds between our `mean_duration_clin*_dead` parameters and survival rates in the simulations.


## Step 1: simplest test

When we solve for time-to-death distribution means using observed survival rates from a simulation run, how close are the results to the `mean_duration_clin*_dead` parameters used in those runs?

Not very. 

This is because survival rates are partly determined by non-CRC death.

The higher the stage, the greater the proportion of deaths due to CRC. So the higher the stage, the closer time-to-death distribution mean is to the `mean_duration_clin*_dead` parameter.

We need to account for non-CRC deaths for this to have any hope of working.

In [2]:
import json

import numpy as np

In [3]:
# read parameters.json
with open("parameters.json") as f:
    params = json.load(f)

In [4]:
# Simulated survival rates from the no screening, no IRR scenario of crcsim-exp-replicate-on-impl-aws

simulated_survival_rates = {
    "stage1": 0.7427,
    "stage2": 0.6927,
    "stage3": 0.5775,
    "stage4": 0.1165,
}

In [5]:
def mean_dist_from_surv_rate(surv_rate, at_year):
    return -1 * at_year / np.log(surv_rate)

In [6]:
for stage in [1, 2, 3, 4]:
    print(f"Stage {stage}")
    simulated_mean = mean_dist_from_surv_rate(
        simulated_survival_rates[f"stage{stage}"], 5
    )
    print("Simulated mean:", simulated_mean)
    actual_mean = params[f"mean_duration_clin{stage}_dead"]
    print("Actual mean:", actual_mean)
    print("Ratio:", simulated_mean / actual_mean)
    print("")

Stage 1
Simulated mean: 16.80880843298729
Actual mean: 135
Ratio: 0.12450969209620216

Stage 2
Simulated mean: 13.61810520068025
Actual mean: 51
Ratio: 0.2670216706015735

Stage 3
Simulated mean: 9.10669120887487
Actual mean: 19
Ratio: 0.4792995373092037

Stage 4
Simulated mean: 2.3257285047335827
Actual mean: 2.7
Ratio: 0.8613809276791047



## Step 2: Account for non-CRC deaths using simulation output

The overall survival function is the product of the CRC survival function and the non-CRC survival function.

So if we can account for non-CRC deaths, we can compute the CRC survival function and the corresponding mean of the time to CRC death distribution.

The easiest way to accomplish this uses the simulation output. We know each agent's expected lifespan, which we use to compute the non-CRC survival function. Then, we combine that with the simulated overall five-year survival rate to get the CRC survival function. 

This works pretty well. The means we get from this method are off by a few percent from the ones used in simulation parameters, but it's within the realm of random variance. I'll eventually try this using output from more runs to see if it gets us closer.

In [7]:
import pandas as pd

In [8]:
outputs = []

# just using a few runs for expedience
for iteration in range(10):
    iteration_name = f"{iteration:03}"

    df = pd.read_csv(
        f"s3://crcsim-crccp-recalibrated-v2/scenarios/no_screening/output_{iteration_name}.csv"
    )
    df["iteration"] = iteration

    outputs.append(df)

output_df = pd.concat(outputs, axis="index")

In [9]:
crc_onsets = output_df[output_df["message"] == "CLINICAL_ONSET"][
    ["iteration", "person_id", "time", "new_state"]
].rename(columns={"time": "onset_time", "new_state": "stage_at_diagnosis"})

In [10]:
lifespans = output_df[output_df["record_type"] == "lifespan"][
    ["iteration", "person_id", "time"]
].rename(columns={"time": "lifespan"})

In [11]:
deaths = output_df[output_df["message"].isin(["OTHER_DEATH", "CRC_DEATH"])][
    ["iteration", "person_id", "time", "message"]
].rename(columns={"time": "death_time", "message": "death_type"})

In [12]:
crc_survival_df = crc_onsets.merge(
    lifespans, on=["iteration", "person_id"], how="left"
).merge(deaths, on=["iteration", "person_id"], how="left")

In [13]:
crc_survival_df["survival_time"] = (
    crc_survival_df["death_time"] - crc_survival_df["onset_time"]
)
crc_survival_df["five_year_survival"] = crc_survival_df["survival_time"] > 5
crc_survival_df["non_crc_survival_time"] = (
    crc_survival_df["lifespan"] - crc_survival_df["onset_time"]
)
crc_survival_df["non_crc_five_year_survival"] = (
    crc_survival_df["non_crc_survival_time"] > 5
)

In [14]:
non_crc_survival_rates = (
    crc_survival_df.groupby("stage_at_diagnosis")["non_crc_five_year_survival"]
    .mean()
    .to_frame()
)

In [15]:
non_crc_survival_rates["simulated_survival"] = list(simulated_survival_rates.values())

In [16]:
# Total survival rate is the product of CRC survival rate and non-CRC survival rate

non_crc_survival_rates["crc_five_year_survival"] = (
    non_crc_survival_rates["simulated_survival"]
    / non_crc_survival_rates["non_crc_five_year_survival"]
)

In [17]:
for index, row in non_crc_survival_rates.reset_index().iterrows():
    print(row["stage_at_diagnosis"])
    simulated_mean = mean_dist_from_surv_rate(row["crc_five_year_survival"], 5)
    print("Simulated mean:", simulated_mean)
    actual_mean = params[f"mean_duration_clin{index + 1}_dead"]
    print("Actual mean:", actual_mean)
    print("Ratio:", simulated_mean / actual_mean)
    print("")

CLINICAL_STAGE1
Simulated mean: 137.14896932307119
Actual mean: 135
Ratio: 1.0159182912820088

CLINICAL_STAGE2
Simulated mean: 49.75101611733958
Actual mean: 51
Ratio: 0.975510119947835

CLINICAL_STAGE3
Simulated mean: 18.138561963703214
Actual mean: 19
Ratio: 0.9546611559843797

CLINICAL_STAGE4
Simulated mean: 2.699679333139679
Actual mean: 2.7
Ratio: 0.9998812344961774



To adjust simulation params to match a different survival rate, we could treat non-CRC survival rate as a constant.

Then we'd find the CRC survival rate that would give us the desired total survival rate.

Finally, we'd find the mean of the time to death distribution that would give us the desired overall survival rate.

Eg:

In [18]:
non_crc_survival_rates["new_survival_rate"] = non_crc_survival_rates[
    "simulated_survival"
] * np.random.uniform(0.9, 0.95, 4)

non_crc_survival_rates["new_crc_five_year_survival"] = (
    non_crc_survival_rates["new_survival_rate"]
    / non_crc_survival_rates["non_crc_five_year_survival"]
)

non_crc_survival_rates["new_mean"] = non_crc_survival_rates[
    "new_crc_five_year_survival"
].apply(lambda x: mean_dist_from_surv_rate(x, 5))
print(non_crc_survival_rates["new_mean"])

stage_at_diagnosis
CLINICAL_STAGE1    42.748904
CLINICAL_STAGE2    29.689019
CLINICAL_STAGE3    13.767720
CLINICAL_STAGE4     2.581565
Name: new_mean, dtype: float64


Small differences in overall survival can lead to large differences in mean time to CRC death, because most deaths are from other causes.

To make this adjustment realistic, we'd also need to account for the differences in non-CRC death rates across the demographic groups we want to simulate. Eg, if African-Americans have a lower CRC survival rate, they probably also have a lower non-CRC survival rate. We don't want to make CRC account for all of the difference. 

## Step 3: Account for non-CRC deaths using death rate params

Death rates by age are part of the simulation parameters. Thus far, we've always used cohorts that have a mix of sexes and races/ethnicities. But to simulate different demographic groups, we could also adjust the cohort so it reflects the death rates of the given group. 

First, we'll try finding the non-CRC death rate from death rate params and comparing it to observed non-CRC death rates. 

### Step 3.1

This whole chunk is just to get overall death rate params for the mix of demographic groups in the cohort.

In [ ]:
import csv

from crcsim.enums import RaceEthnicity, Sex

In [20]:
with open("crcsim/experiment/cohort.csv", mode="r") as input:
    cohort = csv.DictReader(input)

    population = {
        "black_female": 0,
        "black_male": 0,
        "white_female": 0,
        "white_male": 0,
    }

    # Logic from agent.Person
    for person in cohort:
        sex = Sex(person["sex"])
        race_ethnicity = RaceEthnicity(person["race_ethnicity"])

        # Find the appropriate death rate table. We don't have separate tables
        # for all combinations of sex and race_ethnicity, so we'll need to do
        # some imperfect combining of categories.
        if sex == Sex.FEMALE:
            if race_ethnicity == RaceEthnicity.WHITE_NON_HISPANIC:
                population["white_female"] += 1
            elif race_ethnicity in (
                RaceEthnicity.HISPANIC,
                RaceEthnicity.BLACK_NON_HISPANIC,
                RaceEthnicity.OTHER_NON_HISPANIC,
            ):
                population["black_female"] += 1
            else:
                raise ValueError(f"Unexpected race/ethnicity value: {race_ethnicity}")
        elif sex in (Sex.MALE, Sex.OTHER):
            if race_ethnicity == RaceEthnicity.WHITE_NON_HISPANIC:
                population["white_male"] += 1
            elif race_ethnicity in (
                RaceEthnicity.HISPANIC,
                RaceEthnicity.BLACK_NON_HISPANIC,
                RaceEthnicity.OTHER_NON_HISPANIC,
            ):
                population["black_male"] += 1
            else:
                raise ValueError(f"Unexpected race/ethnicity value: {race_ethnicity}")
        else:
            raise ValueError(f"Unexpected sex value: {sex}")

In [21]:
total_population = sum(population.values())
pop_proportions = {k: v / total_population for k, v in population.items()}

In [22]:
death_rates = {
    "black_female": params["death_rate_black_female_rates"],
    "black_male": params["death_rate_black_male_rates"],
    "white_female": params["death_rate_white_female_rates"],
    "white_male": params["death_rate_white_male_rates"],
}

In [23]:
adjusted_death_rates = {
    k: np.array(v) * pop_proportions[k] for k, v in death_rates.items()
}

In [24]:
pop_death_rates = np.sum(list(adjusted_death_rates.values()), axis=0)

### Step 3.2

Now we need the distribution of CRC onsets by stage by age. We'll have to hold this constant for now, although it probably varies by demographic group too. 

In [25]:
def get_onset_distributions(df: pd.DataFrame, max_year: int = 100):
    year_bins = np.arange(max_year + 1)
    stage_distributions = {}
    stages = df["stage_at_diagnosis"].unique()

    for stage in stages:
        stage_data = df[df["stage_at_diagnosis"] == stage]["onset_time"]
        hist, _ = np.histogram(stage_data, bins=year_bins)
        proportions = hist / len(stage_data)
        stage_distributions[stage] = proportions

    return stage_distributions

In [26]:
onset_distributions = get_onset_distributions(crc_survival_df)

### Step 3.3

Next, we calculate the five-year non-CRC survival function for each year.

In [27]:
def n_year_survival(n: int, death_rates: np.ndarray) -> np.ndarray:
    """Calculate the n-year survival rate for a given array of yearly death rates.

    Args:
        n (int): number of years
        death_rates (np.ndarray): array of yearly death rates

    Returns:
        np.ndarray: for each year in death_rates, the probability of surviving n years
    """
    # Pad the death rate array so we get values for the full len(death_rates) years
    # Without padding, sliding_window_view stops when there are fewer than n elements left
    padded_death_rates = np.pad(death_rates, (0, n - 2), constant_values=0)
    # TODO: I think the window for each year should be i+1:i+n+1, not i:i+n
    survival_matrix = np.lib.stride_tricks.sliding_window_view(
        padded_death_rates, window_shape=n
    )
    return np.prod(1 - survival_matrix, axis=1)

In [28]:
five_year_non_crc_survival_rates = n_year_survival(5, pop_death_rates)

### Step 3.4 

Finally, we weight the five-year survival functions by the distributions of CRC onset ages to get five-year non-CRC survival rates by stage of CRC onset.

In [29]:
non_crc_survival_overall = {
    stage: (onset_distributions[stage] * five_year_non_crc_survival_rates).sum()
    for stage in onset_distributions
}

In [30]:
non_crc_survival_rates["computed_non_crc_five_year_survival"] = (
    non_crc_survival_rates.index.map(non_crc_survival_overall)
)

In [31]:
non_crc_survival_rates["non_crc_five_year_survival_ratio"] = (
    non_crc_survival_rates["non_crc_five_year_survival"]
    / non_crc_survival_rates["computed_non_crc_five_year_survival"]
)

In [32]:
non_crc_survival_rates[
    [
        "non_crc_five_year_survival",
        "computed_non_crc_five_year_survival",
        "non_crc_five_year_survival_ratio",
    ]
]

,non_crc_five_year_survival,computed_non_crc_five_year_survival,non_crc_five_year_survival_ratio
stage_at_diagnosis,,,
CLINICAL_STAGE1,0.770276,0.777823,0.990297
CLINICAL_STAGE2,0.765935,0.768888,0.996160
CLINICAL_STAGE3,0.760795,0.761219,0.999444
CLINICAL_STAGE4,0.742456,0.749977,0.989972


The survival rates derived from this approach are very similar to those observed in the simulation. Good sense check. This means that, given CRC survival rates by stage and death rates by year, we can calculate time-to-CRC-death distribution means!

We know we can get the CRC survival rates for various demographics. Just need to confirm that we can get death rates as well. We already have some in the main simulation params, so it seems likely that we could get them for other demographic groups too.

## Step 4: putting it all together. 

To demonstrate how this adjustment would work, we will use the death rates for a single demographic group (black males) from our current parameters.

Recall that we will hold the age-at-onset distribution constant. 

For simplicity, we will modify the simulated survival rates to produce a new target for the demographic group. In production, we would instead use actual survival rates for the target demographic group. [This study](https://pmc.ncbi.nlm.nih.gov/articles/PMC9835097/) doesn't have the exact rates by AJCC stages, but it states that "overall 5-year CRC survival rates for Black Americans have been consistently 6-12% below that of White Americans". So for now, we just modify the simulated rates by roughly that amount.

In [33]:
black_male_survival_targets = {k: v * 0.91 for k, v in simulated_survival_rates.items()}

In [34]:
black_male_survival_targets

{'stage1': 0.675857,
 'stage2': 0.6303570000000001,
 'stage3': 0.525525,
 'stage4': 0.10601500000000001}

In [35]:
black_male_non_crc_survival_rates = n_year_survival(
    5, params["death_rate_black_male_rates"]
)

In [36]:
black_male_non_crc_survival_overall = {
    stage.lower().split("_")[-1]: (
        onset_distributions[stage] * black_male_non_crc_survival_rates
    ).sum()
    for stage in onset_distributions
}

In [37]:
print("Full population non-CRC survival rates:")
print(non_crc_survival_overall)
print("Black males non-CRC surival rates:")
print(black_male_non_crc_survival_overall)

Full population non-CRC survival rates:
{'CLINICAL_STAGE4': 0.7499765462552078, 'CLINICAL_STAGE1': 0.7778232308939725, 'CLINICAL_STAGE2': 0.768887783277248, 'CLINICAL_STAGE3': 0.7612188445712099}
Black males non-CRC surival rates:
{'stage4': 0.7082569301220548, 'stage1': 0.7371309175027159, 'stage2': 0.7275775500631758, 'stage3': 0.7198410573431848}


In [38]:
black_male_df = pd.DataFrame(
    {
        "stage": black_male_survival_targets.keys(),
        "overall_survival": black_male_survival_targets.values(),
        "non_crc_survival": [
            black_male_non_crc_survival_overall[stage]
            for stage in black_male_survival_targets
        ],
    }
)

In [39]:
# Total survival rate is the product of CRC survival rate and non-CRC survival rate

black_male_df["crc_survival"] = (
    black_male_df["overall_survival"] / black_male_df["non_crc_survival"]
)

In [40]:
# Formatting stage for easier printing
non_crc_survival_rates.index = (
    non_crc_survival_rates.index.str.lower().str.split("_").str[-1]
)

In [41]:
for index, row in black_male_df.iterrows():
    stage = row["stage"]
    print(stage.upper())
    print("----------")
    print("Full Population")
    print("\tSimulated survival rate:", simulated_survival_rates[stage])
    full_pop_mean = params[f"mean_duration_clin{index + 1}_dead"]
    print(
        "\tCRC survival rate:",
        non_crc_survival_rates.loc[stage, "crc_five_year_survival"],
    )
    print("\tTime-to-CRC-death dist mean:", full_pop_mean)
    print("Black Males")
    print("\tTarget survival rate:", row["overall_survival"])
    print("\tCRC survival rate:", row["crc_survival"])
    new_target_mean = mean_dist_from_surv_rate(row["crc_survival"], 5)
    print("\tTime-to-CRC-death dist mean:", new_target_mean)
    print("")

STAGE1
----------
Full Population
	Simulated survival rate: 0.7427
	CRC survival rate: 0.9641998346196251
	Time-to-CRC-death dist mean: 135
Black Males
	Target survival rate: 0.675857
	CRC survival rate: 0.9168751221149396
	Time-to-CRC-death dist mean: 57.61430884344316

STAGE2
----------
Full Population
	Simulated survival rate: 0.6927
	CRC survival rate: 0.9043846964913326
	Time-to-CRC-death dist mean: 51
Black Males
	Target survival rate: 0.6303570000000001
	CRC survival rate: 0.866377748935857
	Time-to-CRC-death dist mean: 34.859173789415095

STAGE3
----------
Full Population
	Simulated survival rate: 0.5775
	CRC survival rate: 0.7590741511318243
	Time-to-CRC-death dist mean: 19
Black Males
	Target survival rate: 0.525525
	CRC survival rate: 0.7300569961091502
	Time-to-CRC-death dist mean: 15.891547383493887

STAGE4
----------
Full Population
	Simulated survival rate: 0.1165
	CRC survival rate: 0.15691173736690892
	Time-to-CRC-death dist mean: 2.7
Black Males
	Target survival rate:

## Addendum: Step 5: Digging into the impact of CRC onset age distributions

This whole approach relies on holding the age-at-onset distribution constant.

This got me to thinking - how does this assumption affect the model overall, not just in the context of adjusting CRC survival rates?

### Let's start by thinking about the calibration process:

- Polyp incidence
    - Calibrated to autopsy studies from a range of eras (1963-1992).
        - That decision was pragmatic - there aren't a lot of autopsy studies, and we used all that were available.
    - Takes the average of all available studies for each age group.
    - As such, can be considered somewhere in between the pre-screening and modern eras.
- CRC incidence
    - Calibrated to SEER data from 1975-1979, ie, the pre-screening era.
        - That decision was made because we wanted to calibrate to a time when screening was not a factor.
    - Affects both the transition between polyp and preclinical stages (ie, natural history) and the time to clinical detection. 
        - Clinical detection in the model is driven by symptom onset. Time to symptom onset is the distribution that we calibrate to SEER data.
        - This controls the distribution of CRC onset ages in the absence of screening. Which is appropriate for the pre-screening era calibration targets.
- CRC survival
    - Calibrated to modern SEER data, ie, the screening era.
        - That decision was made because we wanted survival rates to reflect modern treatment options.
        - However, it also means they reflect modern screening strategies, and their effects on the age and stage distributions of CRC onset.
- Non-CRC death
    - Not calibrated, but drawn directly from modern census data.

### Thinking through the implications:

- Aside from the IRR stuff, which we handle elsewhere, it's safe to assume that underlying polyp incidence has not changed with the advent of screening.
- Calibrating CRC incidence to the pre-screening era is a must. We can't have screening affecting our baseline incidence rates. And I think we handle it appropriately by treating the time to clinical detection in the absence of screening.
- Calibrating CRC survival to modern data is more of a tradeoff.
    - CRC survival rates are cause-agnostic. Ie, they include deaths due to CRC and other causes.
        - Actual CRC survival (ie, death due to CRC) reflects modern treatment, which is good.
        - Non-CRC survival reflects modern death tables (ie, rates of death due to other causes), which is also good.
        - But non-CRC survival is also driven by modern age-at-onset, which is bad. 

### How does this affect survival calibration?

- The target overall survival rate is a function of modern CRC death rates, modern non-CRC death rates, and modern age-at-onset distribution.
- The calibrated overall survival rate is a function of calibrated CRC death rates, modern non-CRC death rates, and pre-screening age-at-onset distribution.
- Modern age-at-onset is lower than pre-screening age-at-onset, because CRC is detected younger now thru screening. ([See figure 8](https://www.cancer.org/content/dam/cancer-org/research/cancer-facts-and-statistics/colorectal-cancer-facts-and-figures/colorectal-cancer-facts-and-figures-2023.pdf))
- Therefore, for given non-CRC death rates, non-CRC survival rates are higher with modern age-at-onset.

Let:

$a = \text{overall survival rate}$

$c = \text{CRC survival rate}$, which is determined by the exponential distribution of mean $\mu$ as $c = e^{-\frac{k}{\mu}}$, where $k$ is the k-year survival rate.

$B = \text{non-CRC survival rates} = [b_1, b_2, \ldots, b_n]$

$\theta = \text{age-at-onset distribution} = [\theta_1, \theta_2, \ldots, \theta_n]$, where $\sum (\theta) = 1$.

We are calibrating such that $\hat{a} \approx a$, where $\hat{a}$ is the calibrated survival rate and $a$ is the overall survival rate target.

$c_c (B_m \cdot \theta_p) \approx c_m (B_m \cdot \theta_m)$

Where $c$ denotes calibrated, $p$ denotes pre-screening era, and $m$ denotes modern era.

$(B_m \cdot \theta_p) < (B_m \cdot \theta_m)$
(non-CRC survival rates are higher with modern age-at-onset)

Therefore,

$C_c > C_m$
(CRC survival rates are lower with modern age-at-onset)

So, $\mu_c > \mu_m$ (Our calibrated time-to-CRC-death distribution means are higher than the modern time-to-CRC-death distribution means).

### A better way

Instead of calibrating to $a$, find $c$ by

$a = c (B_m \cdot \theta_m)$

$c = \frac{a}{B_m \cdot \theta_m}$

This would require us knowing $\theta_m$, i.e., finding the modern age-at-onset distribution from SEER or a paper.


## Second Addendum: Step 6: testing computation from relative survival rates

Thus far, we have calibrated our model to overall survival rates (ie, unadjusted survival from death of any cause).

In the literature, relative survival rates are more common. These are estimates of survival from CRC death, adjusted for the competing risk caused by non-CRC death. 

We've avoided calibrating to these thus far because computing relative survival is complex, and we don't include it in the standard model output. 

Now that we have worked out the formula to compute time-to-death distribution means from survival rates, let's see if we can do this for relative survival rates rather than overall. That would avoid the complexity of handling non-CRC deaths with the population death tables. 

In [42]:
# Using relative survival rates that we saved during the 2022 calibration as potential targets.
# These are from 2004-2014, so not exactly the same time as the overall survival rates we ended up calibrating to.
# So we'd expect some difference from that, but we'll start by comparing these survival rates to the time-to-death
# distribution means that produced the calibrated overall survival rates.
#
# # source: https://ascopubs.org/doi/abs/10.1200/JCO.2018.36.4_suppl.587

relative_survival_rate_targets = {
    "stage1": 0.917695925,
    "stage2": 0.823677296,
    "stage3": 0.670109907,
    "stage4": 0.108236099,
}

In [43]:
for stage in [1, 2, 3, 4]:
    print(f"Stage {stage}")
    relative_mean = mean_dist_from_surv_rate(
        relative_survival_rate_targets[f"stage{stage}"], 5
    )
    print("Mean from relative survival computation:", relative_mean)
    actual_mean = params[f"mean_duration_clin{stage}_dead"]
    print("Mean from overall survival calibration:", actual_mean)
    print("Ratio:", relative_mean / actual_mean)
    print("")

Stage 1
Mean from relative survival computation: 58.21455068444863
Mean from overall survival calibration: 135
Ratio: 0.43121889395887875

Stage 2
Mean from relative survival computation: 25.776323999542846
Mean from overall survival calibration: 51
Ratio: 0.5054181176380951

Stage 3
Mean from relative survival computation: 12.490209556990287
Mean from overall survival calibration: 19
Ratio: 0.6573794503679098

Stage 4
Mean from relative survival computation: 2.248767335436035
Mean from overall survival calibration: 2.7
Ratio: 0.8328767909022351



Computing means from relative survival rates gives us much lower means than our calibrated values.

Possible explanations:
1. Underestimation of CRC deaths in model calibration (see prior section)
1. Time difference (Survival could have been somewhat lower in 2004-14. Not this much, but it could account for some of the difference.)
1. Some other factor I'm missing

### Let's try another experiment

For an honest comparison, we need relative survival rates, overall survival rates, and death tables for the same period. 

#### TODO: this remains incomplete. Need age-at-onset dist from same era for honest distribution.

In [ ]:
# 5 Year Survival
# Surveillance, Epidemiology, and End Results (SEER) Program (www.seer.cancer.gov) SEER*Stat Database: Incidence - SEER Research Data, 17 Registries, Nov 2023 Sub (2000-2021) - Linked To County Attributes - Time Dependent (1990-2022) Income/Rurality, 1969-2022 Counties, National Cancer Institute, DCCPS, Surveillance Research Program, released April 2024, based on the November 2023 submission.
# Includes Cases Diagnosed in 2013-2019

target_compare_df = pd.DataFrame(
    {
        "stage": relative_survival_rate_targets.keys(),
        "relative_survival_rate": [0.931, 0.851, 0.698, 0.137],
        "overall_survival_rate": [0.821, 0.721, 0.62, 0.125],
    }
)

In [ ]:
# National Vital Statistics Reports, Volume 72, Number 12 November 7, 2023
# United States Life Tables, 2021

target_compare_death_rates = [
    0.005446,
    0.000403,
    0.000254,
    0.000192,
    0.000161,
    0.000143,
    0.000130,
    0.000119,
    0.000107,
    0.000095,
    0.000090,
    0.000100,
    0.000136,
    0.000205,
    0.000299,
    0.000405,
    0.000513,
    0.000623,
    0.000731,
    0.000837,
    0.000949,
    0.001065,
    0.001170,
    0.001259,
    0.001335,
    0.001406,
    0.001480,
    0.001560,
    0.001651,
    0.001749,
    0.001849,
    0.001947,
    0.002040,
    0.002128,
    0.002216,
    0.002308,
    0.002409,
    0.002519,
    0.002638,
    0.002768,
    0.002916,
    0.003078,
    0.003244,
    0.003410,
    0.003587,
    0.003792,
    0.004036,
    0.004315,
    0.004625,
    0.004959,
    0.005308,
    0.005686,
    0.006118,
    0.006620,
    0.007184,
    0.007766,
    0.008369,
    0.009042,
    0.009795,
    0.010606,
    0.011467,
    0.012333,
    0.013173,
    0.013981,
    0.014798,
    0.015666,
    0.016726,
    0.017853,
    0.019122,
    0.020526,
    0.021919,
    0.023536,
    0.025372,
    0.027616,
    0.029889,
    0.033726,
    0.036933,
    0.041016,
    0.044758,
    0.049530,
    0.054120,
    0.059483,
    0.065401,
    0.072224,
    0.080609,
    0.089139,
    0.099586,
    0.111021,
    0.123484,
    0.137001,
    0.151584,
    0.167229,
    0.183913,
    0.201590,
    0.220190,
    0.239623,
    0.259772,
    0.280504,
    0.301662,
    0.323082,
    1,
]

In [ ]:
# TODO: replace with current onset distributions

onset_distributions = pd.DataFrame(
    {k.lower().split("_")[-1]: v for k, v in onset_distributions.items()}
)

In [54]:
weighted_non_crc_survival = {
    k: sum(v * n_year_survival(5, target_compare_death_rates))
    for k, v in onset_distributions.items()
}

In [55]:
target_compare_df["weighted_non_crc_survival"] = target_compare_df["stage"].map(
    weighted_non_crc_survival
)

In [56]:
target_compare_df["relative_mean"] = target_compare_df["relative_survival_rate"].apply(
    lambda x: mean_dist_from_surv_rate(x, 5)
)

In [57]:
target_compare_df["computed_cause_specific_survival"] = (
    target_compare_df["overall_survival_rate"]
    / target_compare_df["weighted_non_crc_survival"]
)

In [58]:
target_compare_df["overall_mean"] = target_compare_df[
    "computed_cause_specific_survival"
].apply(lambda x: mean_dist_from_surv_rate(x, 5))